In [3]:
from glob import glob

for g in glob(r'C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap13\인공지능을 활용한 뉴미디어아트 전시 공간의 확장 가능성 연구.pdf'):
    print(g)

C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap13\인공지능을 활용한 뉴미디어아트 전시 공간의 확장 가능성 연구.pdf


In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def read_pdf_and_split_text(pdf_path, chunk_size=100, chunk_overlap=30):
    """
    주어진 PDF 파일을 읽고 텍스트를 분할합니다.
    매개변수:
        pdf_path (str): PDF 파일의 경로.
        chunk_size (int, 선택적): 각 텍스트 청크의 크기, 기본값은 100입니다.
        chunk_overlap (int, 선택적): 청크 간의 중첩 크기, 기본값은 30입니다.
    반환값:
        list: 분할된 텍스트 청크의 리스트
    """
    pdf_path= r'C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap13\인공지능을 활용한 뉴미디어아트 전시 공간의 확장 가능성 연구.pdf'
    
    print(f"PDF: {pdf_path} -------------------------")
    pdf_loader = PyPDFLoader(pdf_path)
    data_from_pdf = pdf_loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size, chunk_overlap = chunk_overlap
    )

    splits = text_splitter.split_documents(data_from_pdf)

    print(f"Number of splits:{len(splits)}\n")
    return splits

In [5]:
# [VS Code / Codespaces] Gemini API (text-embedding-004 + Robust Retry)
import os
import time
from glob import glob
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# 1. 환경 변수 로드
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY가 설정되지 않았습니다.")

# 2. 최신 임베딩 모델 설정 (text-embedding-004)
embedding = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004", 
    google_api_key=GOOGLE_API_KEY
)

# 3. 저장 경로 설정 (모델이 바뀌었으므로 경로도 구분하는 것이 좋습니다)
persist_directory = './chroma_store_gemini'

# 4. 벡터 스토어 생성 로직
if os.path.exists(persist_directory):
    print(f"Loading existing Chroma store from {persist_directory}")
    vectorstore = Chroma(
        persist_directory=persist_directory, 
        embedding_function=embedding
    )
else:
    print("Creating new Chroma store...")
    vectorstore = None
    
    # PDF 파일 경로 설정 (본인의 경로에 맞게 수정)
    pdf_files = glob(r'C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap13\인공지능을 활용한 뉴미디어아트 전시 공간의 확장 가능성 연구.pdf') 
    
    for g in pdf_files:
        print(f"Processing file: {g}")
        # read_pdf_and_split_text 함수는 이미 정의되어 있다고 가정
        chunks = read_pdf_and_split_text(g)
        
        # [중요] Rate Limit 방지를 위한 설정
        # 무료 티어는 분당 요청(RPM) 제한이 있으므로 한 번에 조금씩 처리합니다.
        batch_size = 10 
        
        for i in range(0, len(chunks), batch_size):
            batch = chunks[i:i+batch_size]
            
            # 재시도(Retry) 로직: 에러 발생 시 대기 후 재시도
            max_retries = 5
            for attempt in range(max_retries):
                try:
                    if vectorstore is None:
                        vectorstore = Chroma.from_documents(
                            documents=batch,
                            embedding=embedding,
                            persist_directory=persist_directory
                        )
                    else:
                        vectorstore.add_documents(documents=batch)
                    
                    print(f"  - Processed batch {i} ~ {i+len(batch)}")
                    
                    # [핵심] 성공 후 3초 대기 (분당 15회 제한 준수)
                    time.sleep(3) 
                    break  # 성공하면 재시도 루프 탈출
                    
                except Exception as e:
                    # 429(ResourceExhausted) 에러 발생 시
                    if "429" in str(e) or "ResourceExhausted" in str(e):
                        wait_time = (attempt + 1) * 30 # 30초, 60초... 점진적 대기
                        print(f"  ⚠️ Quota exceeded. Waiting {wait_time} seconds...")
                        time.sleep(wait_time)
                    else:
                        print(f"  ❌ Error: {e}")
                        raise e # 다른 에러는 즉시 중단

Creating new Chroma store...
Processing file: C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap13\인공지능을 활용한 뉴미디어아트 전시 공간의 확장 가능성 연구.pdf
PDF: C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap13\인공지능을 활용한 뉴미디어아트 전시 공간의 확장 가능성 연구.pdf -------------------------
Number of splits:241

  - Processed batch 0 ~ 10
  - Processed batch 10 ~ 20
  - Processed batch 20 ~ 30
  - Processed batch 30 ~ 40
  - Processed batch 40 ~ 50
  - Processed batch 50 ~ 60
  - Processed batch 60 ~ 70
  - Processed batch 70 ~ 80
  - Processed batch 80 ~ 90
  - Processed batch 90 ~ 100
  - Processed batch 100 ~ 110
  - Processed batch 110 ~ 120
  - Processed batch 120 ~ 130
  - Processed batch 130 ~ 140
  - Processed batch 140 ~ 150
  - Processed batch 150 ~ 160
  - Processed batch 160 ~ 170
  - Processed batch 170 ~ 180
  - Processed batch 180 ~ 190
  - Processed batch 190 ~ 200
  - Processed batch 200 ~ 210
  - Processed batch 210 ~ 220
  - Processed batch 220 ~ 230
  - Processed b

In [6]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

chunks = retriever.invoke("뉴미디어 아트")

for chunk in chunks:
    print(chunk.metadata)
    print(chunk.page_content)

{'author': '', 'source': 'C:\\Users\\ziclp\\OneDrive\\Desktop\\LLM Assignment\\1week\\PRACTICE1\\chap13\\인공지능을 활용한 뉴미디어아트 전시 공간의 확장 가능성 연구.pdf', 'page_label': '9', 'subject': '', 'creationdate': '2024-12-29T20:56:08+09:00', 'page': 8, 'pdfversion': '1.4', 'creator': '', 'producer': 'Hancom PDF 1.3.0.542; modified using iTextSharp™ 5.5.0 ©2000-2013 iText Group NV (AGPL-version)', 'keywords': '', 'total_pages': 10, 'title': '', 'moddate': '2024-12-30T10:49:17+08:00'}
 예술문화와뉴미디어의결합 전통적인갤러리와가상공간에서함께전시 디지털공간과물리적공간에서함께존재하는하이브리드전시 미술관이라는실재전시실을가상의공간으로확장
{'moddate': '2024-12-30T10:49:17+08:00', 'creator': '', 'author': '', 'page_label': '6', 'producer': 'Hancom PDF 1.3.0.542; modified using iTextSharp™ 5.5.0 ©2000-2013 iText Group NV (AGPL-version)', 'subject': '', 'page': 5, 'title': '', 'total_pages': 10, 'source': 'C:\\Users\\ziclp\\OneDrive\\Desktop\\LLM Assignment\\1week\\PRACTICE1\\chap13\\인공지능을 활용한 뉴미디어아트 전시 공간의 확장 가능성 연구.pdf', 'pdfversion': '1.4', 'keywords': '', 'creationdate': '2

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", # 원본 모델 유지
    api_key=GOOGLE_API_KEY,
    temperature=0.0
)
chat.invoke('예술문화와 뉴미디어에 대해 알려줘')


AIMessage(content='예술문화와 뉴미디어는 서로에게 깊은 영향을 주고받으며 끊임없이 진화하고 있습니다. 뉴미디어의 등장은 예술의 창작, 유통, 향유 방식은 물론, 문화 전반의 지형을 근본적으로 변화시키고 있습니다.\n\n### 뉴미디어의 등장과 예술문화의 변화\n\n뉴미디어는 디지털 기술을 기반으로 하는 모든 형태의 미디어를 총칭하며, 인터넷, 소셜 미디어, 가상현실(VR), 증강현실(AR), 인공지능(AI), 블록체인 등을 포함합니다. 이러한 뉴미디어는 예술문화에 다음과 같은 변화를 가져왔습니다.\n\n1.  **창작 방식의 혁신:**\n    *   **새로운 도구와 재료:** 디지털 페인팅, 3D 모델링, 사운드 디자인 소프트웨어, 프로젝션 맵핑 등 뉴미디어 기술은 예술가들에게 새로운 창작 도구와 재료를 제공합니다.\n    *   **새로운 예술 형식의 탄생:** 미디어 아트, 인터랙티브 아트, 넷 아트(Net Art), 게임 아트, VR/AR 아트, AI 아트 등 뉴미디어를 활용한 완전히 새로운 예술 장르가 등장했습니다.\n    *   **융합과 협업의 증대:** 기술자와 예술가, 과학자와 예술가 등 다양한 분야의 전문가들이 협업하여 복합적인 예술 작품을 만들어내는 경우가 많아졌습니다.\n\n2.  **유통 및 접근성의 확장:**\n    *   **온라인 플랫폼:** 유튜브, 인스타그램, 틱톡 등 소셜 미디어와 온라인 갤러리, 스트리밍 서비스는 예술 작품이 전 세계 관객에게 도달하는 문턱을 낮췄습니다.\n    *   **가상 박물관 및 전시:** VR/AR 기술을 활용하여 실제 박물관에 가지 않고도 작품을 감상하고 전시를 체험할 수 있게 되었습니다.\n    *   **예술의 민주화:** 누구나 쉽게 자신의 작품을 공유하고 피드백을 받을 수 있게 되면서, 예술 창작과 향유의 주체가 확장되었습니다.\n\n3.  **관객 경험의 혁신:**\n    *   **상호작용성(Interactivity):** 관객이 작품에 직접 참여하고 반응함으

In [10]:
# Router 설정
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal # 문자열 리터럴 타입을 지원하는 typing 모듈의 클래스
from pydantic import BaseModel, Field

# Data model
class RouteQuery(BaseModel):
    """사용자 쿼리를 가장 관련성이 높은 데이터 소스로 라우팅합니다."""
    
    datasource: Literal["vectorstore", "casual_talk"] = Field(
        ...,
        description="""
        사용자 질문에 따라 casual_talk 또는 vectorstore로 라우팅합니다.
        - casual_talk: 일상 대화를 위한 데이터 소스. 사용자가 일상적인 질문을 할 때 사용합니다.
        - vectorstore: 사용자 질문에 답하기 위해 RAG로 vectorstore 검색이 필요한 경우 사용합니다.
        """,
    )

In [13]:
# 특정 모델을 structured output (구조화된 출력)과 함께 사용하기 위해 설정
structured_llm_router = chat.with_structured_output(RouteQuery)

router_system = """
당신은 사용자의 질문을 vectorstore 또는 casual_talk으로 라우팅하는 전문가입니다.
- vectorstore에는 인공지능을 활용한 뉴미디어아트와 관련된 문서가 포함되어 있습니다. 이 주제에 대한 질문에는 vectorstore를 사용하십시오.
- 사용자의 질문이 일상 대화에 관련된 경우 casual_talk을 사용하십시오.
"""

# 시스템 메시지와 사용자의 질문을 포함하는 프롬프트 템플릿 생성
route_prompt = ChatPromptTemplate.from_messages([
    ("system", router_system),
    ("human", "{question}"),
])

# 라우터 프롬프트와 구조화된 출력 모델을 결합한 객체
question_router = route_prompt | structured_llm_router


In [15]:
# ✅ 올바른 호출 방식 (.invoke 사용)
print(
    question_router.invoke({
        "question": "뉴미디어아트에서 인공지능의 역할은 무엇인가요?"
    })
)

print(question_router.invoke({"question": "잘 지냈어?"}))

datasource='vectorstore'
datasource='casual_talk'
